In [12]:
from bs4 import BeautifulSoup
import requests
import json
import time
import random

In [13]:
player_summary_url = "http://www.howstat.com/cricket/Statistics/Players/PlayerOverviewSummary.asp?PlayerID="
# player_id = "3600"

In [14]:
# website_html = requests.get(player_summary_url + player_id)
# soup = BeautifulSoup(website_html.content, 'html.parser')

In [15]:
VALID_COUNTRIES = ["australia", "bangladesh", "england", "india", "new zealand", "pakistan", "south africa", "sri lanka", "west indies", "afghanistan", "netherlands"]

match_type_mapping = {
    "BorderedBoxTest": "test",
    "BorderedBoxODI": "odi",
    "BorderedBoxT20": "t20",
    "BorderedBoxIPL": "ipl",
    "BorderedBoxOverall": "overall"
}

# soup.find('span', string="Innings:").find_next('span', string="Innings:").find_next('span', string="Innings:").find_next('td').text.strip()

In [16]:
with open("./howstat_json/howstat_players.json", "r") as outfile:
    player_json_data = json.loads(outfile.read())
    print(len(player_json_data))

6740


In [17]:
ALL_PLAYER_STATS = []

match_index_mapping ={
        "batting": {
            "innings": "Innings:",
            "notOuts": "Not Outs:",
            "agrregate": "Aggregate:",
            "highestScore": "Highest Score:",
            "average": "Average:",
            "50s": "50s:",
            "100s": "100s:",
            "ducks": "Ducks:",
            "4s": "4s:",
            "6s": "6s:",
            "scoringRate": "Scoring Rate"            
        },
        "bowling": {
            "Overs": "Overs:",
            "runsConceded": "Runs Conceded:",
            "wicktes": "Wickets:",
            "bowlingAverage": "Average:",
            # "5Wickets": "5 Wickets in Innings:",
            "bowlingEconomyRate": "Economy Rate:",
            "bowlingStrikeRate": "Strike Rate:",
        },
        "captaincy": {
            "Won/Lost:": "Won/Lost:"
            }
    }


In [18]:
# player_json_data = [{
#                         'name': 'v kohli',
#                         'id': 3600,
#                         'dob': '31/01/1986',
#                         'country': 'India',
#                         'test_matches': 223,
#                         'odi_matches': 123,
#                         't20_matches': 27
#                 }]

# player_id = "3600"



In [19]:
def get_stats(soup, current_title_index, match_title_row, style, metric, value):
    global PLAYER_STAT
    if current_title_index == len(match_title_row):
        return PLAYER_STAT

    if current_title_index == 0:
        target_element_value = soup.find('span', string=value).find_next('td').text.strip()
        next_soup = soup.find('span', string=value)

    else:
        target_element_value = soup.find_next('span', string=value).find_next('td').text.strip()
        next_soup = soup.find_next('span', string=value)

    match_category = match_title_row[current_title_index]['class'][0]

    if match_type_mapping[match_category] == 'odi' or match_type_mapping[match_category] == 'test' or match_type_mapping[match_category] == 't20':
        PLAYER_STAT[match_type_mapping[match_category]][style][metric] = target_element_value


    return get_stats(next_soup, current_title_index + 1, match_title_row, style, metric, value)

In [20]:
counter = 0

for player in player_json_data[6576:]:
    if player['country'].lower() not in VALID_COUNTRIES:
        continue
    
    PLAYER_STAT = {
        "odi": {
            "batting": {},
            "bowling": {},
            "captaincy": {}
        },
        "test": {
            "batting": {},
            "bowling": {},
            "captaincy": {}
        },
        "t20": {
            "batting": {},
            "bowling": {},
            "captaincy": {}
        },
    }

    name = False
    player_id = str(player['id'])
    PLAYER_STAT['playerId'] = player['id']
    PLAYER_STAT['country'] = player['country']
    PLAYER_STAT['totalTestMatches'] = player['test_matches']
    PLAYER_STAT['totalODIMatches'] = player['odi_matches']
    PLAYER_STAT['totalT20Matches'] = player['t20_matches']
    PLAYER_STAT['dob'] = player['dob']

    website_html = requests.get(player_summary_url + player_id)
    soup = BeautifulSoup(website_html.content, 'html.parser')

    try:
        top_tr = soup.find('table', attrs={'border': "0", 'width': "100%", 'cellpadding': "2", 'cellspacing': "0"}).find_all('tr')
        PLAYER_STAT['name'] = top_tr[2].find('td', attrs={'width': '450'}).text.strip()
        PLAYER_STAT['battingStyle'] = top_tr[5].find('td').find_next('td').text.strip()
        PLAYER_STAT['bowlingStyle'] = top_tr[6].find('td').find_next('td').text.strip()
        name = True
    except Exception as e:
        print("ERROR ERROR ERROR ERROR ERROR ERROR", player_id, e)

    match_title_row = soup.find_all('table', attrs={'border':'0', 'cellpadding':'1', 'width':'244'})

    if len(match_title_row) == 0:
        continue

    for key1 in match_index_mapping.keys():
        for key2 in match_index_mapping[key1].keys():
            style = key1
            metric = key2
            value = match_index_mapping[key1][key2]
            get_stats(soup, 0, match_title_row, style, metric, value)

    ALL_PLAYER_STATS.append(PLAYER_STAT)
    time.sleep(random.randint(1, 20))
    counter += 1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
ERROR ERROR ERROR ERROR ERROR ERROR 24 'NoneType' object has no attribute 'find_all'
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
ERROR ERROR ERROR ERROR ERROR ERROR 530 'NoneType' object has no attribute 'find_all'
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
ERROR ERROR ERROR ERROR ERROR ERROR 3 'NoneType' object has no attribute 'find_all'
62
63
64
65
66
ERROR ERROR ERROR ERROR ERROR ERROR 934 'NoneType' object has no attribute 'find_all'
67
68
69
70
71
72
ERROR ERROR ERROR ERROR ERROR ERROR 25 'NoneType' object has no attribute 'find_all'
73


In [21]:
ALL_PLAYER_STATS 

[{'odi': {'batting': {}, 'bowling': {}, 'captaincy': {}},
  'test': {'batting': {'innings': '4',
    'notOuts': '1',
    'agrregate': '204',
    'highestScore': '134*',
    'average': '68.00',
    '50s': '1',
    '100s': '1',
    'ducks': '0',
    '4s': '11',
    '6s': '0',
    'scoringRate': 'N/A'},
   'bowling': {'Overs': '0.0',
    'runsConceded': '0',
    'wicktes': '0',
    'bowlingAverage': '68.00',
    'bowlingEconomyRate': '',
    'bowlingStrikeRate': 'N/A'},
   'captaincy': {'Won/Lost:': ''}},
  't20': {'batting': {}, 'bowling': {}, 'captaincy': {}},
  'playerId': 1930,
  'country': 'england',
  'totalTestMatches': 4,
  'totalODIMatches': 0,
  'totalT20Matches': 0,
  'dob': '14/12/1853',
  'name': 'Henry Wood',
  'battingStyle': 'Right Handed',
  'bowlingStyle': 'Does Not Bowl | Unknown'},
 {'odi': {'batting': {'innings': '23',
    'notOuts': '14',
    'agrregate': '152',
    'highestScore': '43*',
    'average': '29.45',
    '50s': '0',
    '100s': '0',
    'ducks': '3',
    

In [ ]:
#######################################

DONE - 0:1500 

In [ ]:
len(player_json_data[4000:])

In [ ]:
for i in range(len(player_json_data)):
    if player_json_data[i]['id'] == 1930:
        print(i)